In [1]:
#import libraries
import pandas as pd
import tensorflow as tf
from keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
from sklearn import metrics
import time


In [2]:
dnames = ['all_10', 'all_20', 'all_30', 'all_40', 'all_50', 'all_60', 'all_70', 'all_80', 'all_90', 'all_100']
tle = []

for a in range(len(dnames)):
    x = np.load('../data/training_data/' + dnames[a] + '_input_x.npy')
    y = np.load('../data/training_data/' + dnames[a] + '_input_y.npy')

    result = np.where(y==1) # Identify take over instances
    all_indices = list(range(y.shape[0]))
    ntc_index = [i for i in all_indices if i not in result[0]] # Identify non-take over instances
    tc_index = list(result[0])

    # Take a subset of the non-take over instances such that class ratio is 1:1
    tc_idx_size = len(tc_index)
    random.shuffle(ntc_index)
    ntc_data_idx = ntc_index[:tc_idx_size]

    # Combine and shuffle the indices
    idx = ntc_data_idx+tc_index
    random.shuffle(idx)

    X = x[idx]
    Y = y[idx]
    Y = Y.reshape(-1,1)

    x_t, x_test, y_t, y_test = train_test_split(X, Y, test_size=0.25, shuffle=True)
    x_train, x_val, y_train, y_val = train_test_split(x_t, y_t, test_size=0.25, shuffle=True)

    x_train = [x_train[:,:,:18], x_train[:,:,18:21], x_train[:,:,21:42], x_train[:,:,42:]]
    x_val = [x_val[:,:,:18], x_val[:,:,18:21], x_val[:,:,21:42], x_val[:,:,42:]]
    x_test = [x_test[:,:,:18], x_test[:,:,18:21], x_test[:,:,21:42], x_test[:,:,42:]]

    tf.random.set_seed(0)

    classifier = models.Model()

    #inputs
    dd = layers.Input((X.shape[1], 18, 1))
    ac = layers.Input((X.shape[1], 3, 1))
    gz = layers.Input((X.shape[1], 21, 1))
    gy = layers.Input((X.shape[1], 3, 1))

    #driving data block
    out_dd = layers.Conv2D(8, (3,3), activation='relu', padding='same')(dd)
    out_dd = layers.MaxPool2D((2,2))(out_dd)
    out_dd = layers.Conv2D(16, (3,3), activation='relu', padding='same')(out_dd)
    out_dd = layers.MaxPool2D((2,2))(out_dd)
    out_dd = layers.Flatten()(out_dd)

    #accelerometer block
    out_ac = layers.Conv2D(8, (3,3), activation='relu', padding='same')(ac)
    out_ac = layers.Conv2D(16, (3,3), activation='relu', padding='same')(out_ac)
    out_ac = layers.MaxPool2D((2,2))(out_ac)
    out_ac = layers.Flatten()(out_ac)

    #gaze block
    out_gz = layers.Conv2D(8, (3,3), activation='relu', padding='same')(gz)
    out_gz = layers.MaxPool2D((2,2))(out_gz)
    out_gz = layers.Conv2D(16, (3,3), activation='relu', padding='same')(out_gz)
    out_gz = layers.MaxPool2D((2,2))(out_gz)
    out_gz = layers.Flatten()(out_gz)

    #gyroscope block
    out_gy = layers.Conv2D(8, (3,3), activation='relu', padding='same')(gy)
    out_gy = layers.Conv2D(16, (3,3), activation='relu', padding='same')(out_gy)
    out_gy = layers.MaxPool2D((2,2))(out_gy)
    out_gy = layers.Flatten()(out_gy)

    #join the flattened output layers
    output = layers.concatenate([out_dd, out_ac, out_gz, out_gy], axis = 1)
    
    #attention layer
    att_layer = layers.Dense(output.shape[1], activation ='softmax', name = 'attention_vec')(output)
    attention_mul = layers.multiply([output,att_layer], name = 'attention_mul')

    #dense layers
    output = layers.Dense(16, activation='tanh')(attention_mul)
    output = layers.Dense(8, activation='tanh')(output)
    output = layers.Dense(1, activation='sigmoid')(output)

    #model
    classifier = models.Model(inputs = [dd, ac, gz, gy], outputs = output)
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    start = time.time()

    history = classifier.fit(x_train, y_train, epochs=20, validation_data=(x_val, y_val), verbose=0)

    end = time.time()

    plt.plot(history.history['accuracy'], label='train accuracy')
    plt.plot(history.history['val_accuracy'], label='validation accuracy')
    plt.plot(history.history['loss'], label='train loss')
    plt.plot(history.history['val_loss'], label='validation loss')
    plt.xlabel('# of epochs')
    plt.ylabel('accuracy / loss')
    plt.ylim([0,1])
    plt.legend(loc='center right')
    plt.title(dnames[a])
    plt.savefig('../data/results/figures/multi_modal/' + dnames[a] + '.png', dpi=800)
    plt.close()

    y_pred = classifier.predict(x_test, verbose=0)
    for i in y_pred:
        if i[0] >= 0.5:
            i[0] = 1
        else:
            i[0] = 0
    cf = metrics.confusion_matrix(y_test, y_pred)
    total=sum(sum(cf))
    accuracy = (cf[0,0]+cf[1,1])/total
    specificity = cf[0,0]/(cf[0,0]+cf[0,1])
    sensitivity = cf[1,1]/(cf[1,0]+cf[1,1]) #recall
    precision = cf[1,1]/(cf[0,1]+cf[1,1])
    f1 = 2*(1/((1/precision)+(1/sensitivity)))
    ti = end - start
    lt = [dnames[a], accuracy, specificity, sensitivity, precision, f1, ti]
    tle.append(lt)

metric = pd.DataFrame(tle, columns=['datatype', 'accuracy', 'specificity', 'sensitivity', 'precision', 'f1-score', 'train_time'])
metric.to_excel('../data/results/accuracy_tables/multi_modal.xlsx', index=False)


2023-02-25 20:07:24.446300: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-25 20:07:24.447408: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M2

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-25 20:07:24.751774: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-25 20:07:25.056155: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-25 20:07:27.731335: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-25 20:07:54.584233: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-25 20:07:57.873630: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-25 20:08:00.722693: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-25 20:08:48.570370: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114